# Model recovery attack: `credit-card`

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mra
from mra import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

df = pd.read_excel('../datasets/credit-card.xls', header=1, index_col=0).sample(frac=1)
min_values = df.drop(columns=["default payment next month"]).describe().transpose()['min'].to_numpy()
max_values = df.drop(columns=["default payment next month"]).describe().transpose()['max'].to_numpy()
x = df.drop(columns=["default payment next month"]).to_numpy()
x = (x-min_values)/(max_values-min_values)
y = df["default payment next month"].to_numpy().reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

mra = mra(train_ds)

2022-02-17 01:13:00.483858: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-17 01:13:00.483911: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: xgpg2
2022-02-17 01:13:00.483921: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: xgpg2
2022-02-17 01:13:00.484033: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.103.1
2022-02-17 01:13:00.484060: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-02-17 01:13:00.484067: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.103.1
2022-02-17 01:13:00.484533: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in per

In [3]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    output = tf.keras.layers.Dense(64, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    # x = tf.keras.layers.Dense(64, activation="relu")(xin)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(xin)
    return tf.keras.Model(xin, output)

In [4]:
log_train = mra.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.binary_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=1000
    )

Iteration 0001: Training loss: 0.7072
Iteration 1001: Training loss: 0.3672
Iteration 2001: Training loss: 0.4957
Iteration 3001: Training loss: 0.3395
Iteration 4001: Training loss: 0.3932


In [5]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(512, activation=act)(xin)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    x = tf.keras.layers.Dense(64, activation=act)(x)
    # x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(23, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [6]:
log_attack = mra.attack(
    attack_iter=100,
    make_generator=make_generator,
    input_noise="normal",
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.2307, RG-normal: 0.2125, reconstruction validation: 0.0715
Iteration 0002: RG-uniform: 0.2501, RG-normal: 0.2270, reconstruction validation: 0.0670
Iteration 0003: RG-uniform: 0.2398, RG-normal: 0.2085, reconstruction validation: 0.0613
Iteration 0004: RG-uniform: 0.2297, RG-normal: 0.1926, reconstruction validation: 0.0559
Iteration 0005: RG-uniform: 0.2444, RG-normal: 0.2097, reconstruction validation: 0.0446
Iteration 0006: RG-uniform: 0.2352, RG-normal: 0.2169, reconstruction validation: 0.0553
Iteration 0007: RG-uniform: 0.2189, RG-normal: 0.2071, reconstruction validation: 0.0471
Iteration 0008: RG-uniform: 0.2409, RG-normal: 0.2207, reconstruction validation: 0.0593
Iteration 0009: RG-uniform: 0.2306, RG-normal: 0.2069, reconstruction validation: 0.0553
Iteration 0010: RG-uniform: 0.2343, RG-normal: 0.2153, reconstruction validation: 0.0593
Iteration 0011: RG-uniform: 0.2439, RG-normal: 0.2138, reconstruction validation: 0.0465
Iteration 0012: RG-un

In [18]:
np.savetxt("./log/credit-4.csv", log_attack, delimiter=",")